In [1]:
import os
os.chdir("..")

In [2]:
import django
# In case that we need it later
#from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'billboard.settings')
# We need to enable the following because jupyter is running an async loop
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
import json
import http.client
from rankings.models import Song

In [4]:
from django.conf import settings
api_key = settings.API_KEY

conn = http.client.HTTPSConnection("billboard-api2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': api_key,
    'x-rapidapi-host': "billboard-api2.p.rapidapi.com"
}

conn.request("GET", "/hot-100?date=2024-06-05&range=1-100", headers=headers) # Placeholder date

res = conn.getresponse()
data = res.read()

songs = (data.decode("utf-8"))
songs_data = json.loads(songs)
song_list = songs_data.get('response', [])

In [5]:
from django.utils.dateparse import parse_date

def process_song_data(songs_data):
    for song_key, song_info in songs_data.items():
        title = song_info.get('title')
        artist = song_info.get('artist')
        rank = int(song_info.get('rank'))
        peak_position = int(song_info.get('peak position'))
        weeks_on_chart = int(song_info.get('weeks on chart'))

        chart_date = parse_date('2024-06-05')  # Placeholder date

        song, created = Song.objects.update_or_create(
            title=title,
            artist=artist,
            chart_date=chart_date,
            defaults={
                'rank': rank,
                'album_name': '',
                'peak_position': peak_position,
                'weeks_on_chart': weeks_on_chart,
                'label': '',
                'genre': ''
            }
        )
        if created:
            print(f"Added new song: {title} by {artist}")
        else:
            print(f"Updated song: {title} by {artist}")

In [6]:
song_l = songs_data.get('content', [])
print(song_l)
process_song_data(song_l)

{'1': {'rank': '1', 'title': 'I Had Some Help', 'artist': 'Post Malone Featuring Morgan Wallen', 'weeks at no.1': '3', 'last week': '1', 'peak position': '1', 'weeks on chart': '3', 'detail': 'same'}, '2': {'rank': '2', 'title': 'Not Like Us', 'artist': 'Kendrick Lamar', 'last week': '2', 'peak position': '1', 'weeks on chart': '4', 'detail': 'same'}, '3': {'rank': '3', 'title': 'Million Dollar Baby', 'artist': 'Tommy Richman', 'last week': '3', 'peak position': '2', 'weeks on chart': '5', 'detail': 'same'}, '4': {'rank': '4', 'title': 'A Bar Song (Tipsy)', 'artist': 'Shaboozey', 'last week': '4', 'peak position': '3', 'weeks on chart': '7', 'detail': 'same'}, '5': {'rank': '5', 'title': 'Espresso', 'artist': 'Sabrina Carpenter', 'last week': '6', 'peak position': '4', 'weeks on chart': '7', 'detail': 'up'}, '6': {'rank': '6', 'title': 'Pink Skies', 'artist': 'Zach Bryan', 'last week': 'None', 'peak position': '6', 'weeks on chart': '1', 'detail': 'new'}, '7': {'rank': '7', 'title': 'L